In [1]:
import confirm.outlaw.nb_util as nb_util

nb_util.setup_nb()
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
from numpy import sqrt
import jax
import jax.numpy as jnp
from itertools import product
from scipy.special import logit
import confirm.mini_imprint.binomial as binomial

In [32]:
n_arm_samples = 50
nsims = 10000
d = 2
center_p = 0.2
theta_tiles = np.full((1, d), logit(center_p))
radii = np.array([0.2])
hypercube = np.array(list(product((1, -1), repeat=d)))
tile_corners = theta_tiles + hypercube[None, :] * radii
is_null_per_arm = np.full_like(theta_tiles, 1, dtype=bool)

delta = 0.01
holderp = 1.2
holderq = 6
thresh = 20


In [33]:
np.random.seed(17)
uniforms = np.random.uniform(size=(nsims, n_arm_samples, d))
typeI_sum, typeI_score = binomial.binomial_accumulator(lambda x: x[..., 0] >= thresh)(
    theta_tiles, is_null_per_arm, uniforms
)

typeI_est, typeI_CI = binomial.zero_order_bound(typeI_sum, nsims, delta, 1.0)
typeI_bound = typeI_est + typeI_CI

hob = binomial.holder_odi_bound(
    typeI_bound, theta_tiles, tile_corners, n_arm_samples, holderq
)
hob

DeviceArray([0.12011727], dtype=float64)

In [26]:
binomial.upper_bound(
    theta_tiles,
    radii,
    tile_corners,
    np.full_like(radii, nsims),
    n_arm_samples,
    typeI_sum,
    typeI_score,
    0.01,
)


(DeviceArray([0.50371321], dtype=float64),
 DeviceArray([0.0011], dtype=float64),
 array([0.00117658]),
 DeviceArray([0.00326376], dtype=float64),
 array([0.11423109]),
 array([0.38394178]))